In [1]:
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
import time
import optuna
from sklearn.metrics import f1_score, classification_report
from machine_learning.Model_opt import Combined_xgb_logreg

In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_1h_train.csv'))
df

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,1602509400,0,2020-10-12 13:30:00,120.059997,121.330101,119.284500,120.919998,48082766.0,NaN,NaN,NaN,124.314102,NaN,1,0
1,1602513000,0,2020-10-12 14:30:00,120.919998,122.639999,120.769996,122.529998,27028067.0,120.919998,NaN,NaN,124.419998,NaN,0,0
2,1602516600,0,2020-10-12 15:30:00,122.525001,123.629997,122.290000,123.261001,35722193.0,122.529998,120.919998,NaN,123.757202,NaN,0,0
3,1602520200,0,2020-10-12 16:30:00,123.260002,124.000000,123.040000,123.930000,22891159.0,123.261001,122.529998,120.919998,124.430000,NaN,0,0
4,1602523800,0,2020-10-12 17:30:00,124.089996,125.180000,123.910003,125.050003,31443470.0,123.930000,123.261001,122.529998,123.080001,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1609432200,0,2020-12-31 16:30:00,132.529998,132.800003,131.720001,132.619995,13538948.0,132.529998,132.884994,133.679992,NaN,50.983206,0,0
391,1609435800,0,2020-12-31 17:30:00,132.619995,132.717605,132.310104,132.560806,7474176.0,132.619995,132.529998,132.884994,NaN,50.783652,0,0
392,1609439400,0,2020-12-31 18:30:00,132.565002,132.889999,132.009994,132.764999,9408857.0,132.560806,132.619995,132.529998,NaN,51.463321,0,0
393,1609443000,0,2020-12-31 19:30:00,132.770004,133.179992,132.565994,132.800003,10147999.0,132.764999,132.560806,132.619995,NaN,51.582189,0,0


**Entrenamos nuestros modelos para compra**

In [ ]:
close_data = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [3]:

XGBOptimizer(x_train, y_train, x_test, y_test).xgb_optuna()

In [8]:
import xgboost as xgb

# Define the hyperparameters
hyperparameters = {
    'n_estimators': 994,
    'max_depth': 4,
    'max_leaves': 6,
    'learning_rate': 0.1048790005261305,
    'booster': 'dart',
    'gamma': 0.40711115034257384,
    'reg_alpha': 0.975092039533156,
    'reg_lambda': 0.987249221643742
}

# Create the XGBoost model
model = xgb.XGBClassifier(**hyperparameters)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       108
           1       0.00      0.00      0.00        11

    accuracy                           0.91       119
   macro avg       0.45      0.50      0.48       119
weighted avg       0.82      0.91      0.86       119


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
REGLOGOptimizer(x_train, y_train, x_test, y_test).reglog_optuna()

[I 2024-03-06 21:46:50,926] A new study created in memory with name: no-name-92957511-ae85-4dc7-b148-e8647bd9bb4b
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-06 21:46:51,018] Trial 0 finished with value: 0.0 and parameters: {'C': 0.9954640533367793, 'fit_intercept': False, 'l1_ratio': 0.6295501836449665}. Best is trial 0 with value: 0.0.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-06 21:46:51,094] Trial 1 finished with value: 0.0 and parameters: {'C': 4.1256642538634285, 'fit_intercept': True, 'l1_ratio': 0.702

Accuracy: 0.0
Best hyperparameters: {'C': 0.9954640533367793, 'fit_intercept': False, 'l1_ratio': 0.6295501836449665}
Execution time: 0.12603814999262491 minutes


{'C': 0.9954640533367793,
 'fit_intercept': False,
 'l1_ratio': 0.6295501836449665}

In [10]:
from sklearn.linear_model import LogisticRegression

# Define the hyperparameters
hyperparameters = {'C': 0.9954640533367793, 'fit_intercept': False, 'l1_ratio': 0.6295501836449665}

# Create the XGBoost model
model = LogisticRegression(**hyperparameters)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        96
           1       0.00      0.00      0.00         8

    accuracy                           0.92       104
   macro avg       0.46      0.50      0.48       104
weighted avg       0.85      0.92      0.89       104


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

# Optuna parameters for xgboost and Logistic Regression

In [4]:
combined_optimizer = Combined_xgb_logreg(x_train, y_train, x_test, y_test)
combined_optimizer.combined_optuna()

[I 2024-03-07 01:48:57,099] A new study created in memory with name: no-name-9029c20f-bae2-44a7-b103-d7dc4ecd3fd9
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:48:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-07 01:48:57,234] Trial 0 finished with value: 0.0 and parameters: {'xgb_n_estimators': 218, 'xgb_max_depth': 3, 'xgb_max_leaves': 4, 'xgb_learning_rate': 0.09379131701752791, 'xgb_booster': 'gblinear', 'xgb_gamma': 0.9128520641228278, 'xgb_reg_alpha': 0.577027958265575, 'xgb_reg_lambda': 0.5545544635215536, 'reglog_C': 6.52981731525588

F1 Score: 0.0
Best hyperparameters: {'xgb_n_estimators': 218, 'xgb_max_depth': 3, 'xgb_max_leaves': 4, 'xgb_learning_rate': 0.09379131701752791, 'xgb_booster': 'gblinear', 'xgb_gamma': 0.9128520641228278, 'xgb_reg_alpha': 0.577027958265575, 'xgb_reg_lambda': 0.5545544635215536, 'reglog_C': 6.529817315255884, 'reglog_fit_intercept': True, 'reglog_l1_ratio': 0.953820904767933}
Execution time: 0.4291435162226359 minutes


{'xgb_n_estimators': 218,
 'xgb_max_depth': 3,
 'xgb_max_leaves': 4,
 'xgb_learning_rate': 0.09379131701752791,
 'xgb_booster': 'gblinear',
 'xgb_gamma': 0.9128520641228278,
 'xgb_reg_alpha': 0.577027958265575,
 'xgb_reg_lambda': 0.5545544635215536,
 'reglog_C': 6.529817315255884,
 'reglog_fit_intercept': True,
 'reglog_l1_ratio': 0.953820904767933}